In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset('squad', split='train')

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
df = data.to_pandas()
df.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [4]:
df.drop_duplicates(inplace=True, keep='first', subset='context')

In [6]:
df.shape

(18891, 5)

## Embedding 
Documentation: https://platform.openai.com/docs/guides/embeddings/what-are-embeddings?lang=python

In [12]:
import openai
api_key = open('key.txt').read()
client = OpenAI(api_key = api_key)

In [14]:
openai.api_key = api_key
var_model = 'text-embedding-3-small'
response = client.embeddings.create(
    input='I love upskilling myself',
    model= var_model)

In [20]:
print(response.data[0].embedding)

[-0.013762670569121838, -0.009572479873895645, -0.006927734240889549, 0.0007865530205890536, 0.014476501382887363, -0.0006183565128594637, 0.005425118841230869, 0.09987929463386536, -0.0251268669962883, -0.006720723118633032, 0.02191462554037571, -0.000728108047042042, -0.057363495230674744, 0.01184960175305605, 0.08440343290567398, 0.016960635781288147, -0.021729029715061188, -0.016075484454631805, 0.04839777201414108, 0.01940193772315979, 0.001905929995700717, -0.037033576518297195, 0.037290554493665695, 0.009808043949306011, 0.028196342289447784, -0.008002050220966339, 0.028681747615337372, 0.009122765623033047, -0.014890523627400398, 0.00998650211840868, 0.006556541658937931, -0.03848979249596596, -0.08640215545892715, -0.004796947352588177, 0.04017443209886551, 0.0020290659740567207, -0.00499325105920434, -0.004375786520540714, 0.007070500403642654, 7.316772098420188e-05, 0.019773131236433983, -0.06258874386548996, 0.02230009436607361, -0.028267724439501762, -0.005660683382302523,

In [24]:
print(len(response.data[0].embedding))

1536


In [22]:
def get_embedding(text, model=var_model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

## Pinecone

In [23]:
from pinecone import Pinecone
var_pc_api = open('pinecone_key.txt').read()
pc = Pinecone(api_key=var_pc_api)

In [25]:
from pinecone import Pinecone, PodSpec

pc.create_index(
  name="chat-bot",
  dimension=1536,  #embedding size
  metric="dotproduct",
  spec=PodSpec(
    environment="gcp-starter"
  )
)

In [26]:
index = pc.Index("chat-bot")

In [27]:
from tqdm.auto import tqdm
import time

In [32]:
df.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."


In [35]:
embed = OpenAIEmbeddings(
    model = var_model,
    openai_api_key= api_key)

In [41]:
from tqdm.auto import tqdm

count = 0  # we'll use the count to create unique IDs
batch_size = 100  # process everything in batches of 32
for i in tqdm(range(0, len(df['context']), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df['context']))
    # get batch of lines and IDs
    lines_batch = df[i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    # res = client.embeddings.create(input=lines_batch, model=MODEL)
    meta_data = [{"titile" : row['title'], 
              "context": row['context']} 
             for i, row in lines_batch.iterrows()]

    docs = lines_batch['context'].tolist()
    
    emb_vectors = embed.embed_documents(docs)
    
    # prep metadata and upsert batch
    to_upsert = zip(ids_batch, emb_vectors, meta_data)
    # upsert to Pinecone
#     print(list(to_upsert))
    index.upsert(vectors=to_upsert)
    time.sleep(10) # 8s for 50 data points



  0%|          | 0/189 [00:00<?, ?it/s]

Now the embeddings have been pushed to Pinecone. It's time to build an agent using the values. 

In [50]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.18891,
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891}

In [54]:
index

In [63]:
from langchain.vectorstores import Pinecone
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone as PineconeClient

pinecone = PineconeClient(api_key=var_pc_api)

db = pinecone.from_existing_index(index_name='chat-bot', embedding=embed, text_key = 'text')


# vectorstore = Pinecone(index='', embed.embed_query, "text")

AttributeError: 'Pinecone' object has no attribute 'from_existing_index'

In [57]:
query = "president of Notre Dame"

docs = db.similarity_search(query)


NameError: name 'db' is not defined

In [48]:
vectorstore.similarity_search(query, k=3)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


[]